In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:

# Dataset Path
dataset_path = "/home/stu15/s15/pj2196/.cache/kagglehub/datasets/jjangdol/emg-data-for-gestures-data-set/versions/2/EMG/Edited_label"
print("Files in dataset:", os.listdir(dataset_path))

# Load data
files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
print(f"Found {len(files)} files in dataset: {files}")

data = []
for i in range(8): 
    file_path = os.path.join(dataset_path, f"{i}.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        data.append(df)

df = pd.concat(data)
columns = ['time', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'gesture']
df.columns = columns
df.drop(['time'], axis=1, inplace=True)

# Feature extraction
emg_features = ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8']
X_emg = df[emg_features].values
y = df['gesture'].values

# Label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normalize EMG signals
scaler = StandardScaler()
X_emg = scaler.fit_transform(X_emg)

# Reshape data for CNN (samples, timesteps=1, features)
X_emg = X_emg.reshape(X_emg.shape[0], 1, X_emg.shape[1])

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_emg, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create Dataset class
class EMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create DataLoader
train_dataset = EMGDataset(X_train_tensor, y_train_tensor)
val_dataset = EMGDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define CNN model
class EMGModel(nn.Module):
    def __init__(self, num_classes):
        super(EMGModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(128 * 8, 128)
        self.drop1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.drop2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return x

# Model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(np.unique(y))
model = EMGModel(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.5)


Files in dataset: ['4.csv', '3.csv', '2.csv', '5.csv', '6.csv', '1.csv', '0.csv', '7.csv']
Found 8 files in dataset: ['4.csv', '3.csv', '2.csv', '5.csv', '6.csv', '1.csv', '0.csv', '7.csv']


In [4]:

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.4f}")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

Epoch [1/3], Loss: 1.6967, Accuracy: 0.6431
Epoch [2/3], Loss: 1.3416, Accuracy: 0.6431


Fine tuning